In [141]:
import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String, BigInteger, Sequence, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
import os
from libv3.utils import *
import pandas as pd
import json
import shutil

# Define the base for the declarative model
Base = declarative_base()

# Define the Event class which will be mapped to the events table in the database
class Event(Base):
    __tablename__ = 'events'
    
    id: Mapped[int] = mapped_column(primary_key=True)
    
    name: Mapped[int] = mapped_column(nullable=False)
    timestamp: Mapped[int] = mapped_column(BigInteger, nullable=False)
    file_number: Mapped[int] = mapped_column(ForeignKey("file_config.id")) 

    config: Mapped["File_config"] = relationship(back_populates="events")
    


class File_config(Base):
    __tablename__ = 'file_config'

    id: Mapped[int] = mapped_column(primary_key=True)
    unique_name: Mapped[str] = mapped_column(String(50), nullable=False, unique=True)
    code_base: Mapped[str] = mapped_column(String(50), nullable=False)
    version: Mapped[int] = mapped_column(Integer, nullable=False)
    behaviour: Mapped[str] = mapped_column(String(50), nullable=False)
    trial_num: Mapped[int] = mapped_column(Integer, nullable=False)

    events: Mapped[List["Event"]] = relationship(back_populates="config")


def get_or_create(code_base, version, behaviour, trial_num):
    '''
    Function to get or create a new file_config entry in
    the database. This function will return the existing
    entry if it exists, otherwise it will create a new entry
    and return it.

    Args:
        code_base (str): The code base used to generate the data
        version (int): The version of the code base
        behaviour (str): The behaviour being tested
        trial_num (int): The trial number of the data

    Returns:
        File_config: The file_config entry
        bool: False if the entry was created, True if the entry already existed
    '''
    with Session() as session:
        instance = session.query(File_config).filter_by(unique_name=f'{code_base}_{version}_{behaviour}_{trial_num}').first()
        if instance:
            return instance, True  # Return existing entry
        else:
            new_config = File_config(code_base=code_base,version=version, behaviour=behaviour, trial_num=trial_num, unique_name=f'{code_base}_{version}_{behaviour}_{trial_num}')
            session.add(new_config)
            session.commit()
            return new_config, False  # New config added
        
def check_event_exists(name, timestamp, unique_name):
    '''
    Function to check if an event already exists in the database

    Args:
        name (str): The name of the event
        timestamp (int): The timestamp of the event
        file_number (int): The file number of the event

    Returns:
        bool: True if the event exists, False if it does not
    '''
    with Session() as session:
        file_number = session.query(File_config).filter_by(unique_name=unique_name).first().id
        instance = session.query(Event).filter_by(name=name, timestamp=timestamp, file_number=file_number).first()
        if instance:
            return True
        else:
            return False



/var/folders/nw/443sknns78l256x46fjj6r0c0000gn/T/ipykernel_2062/3485431635.py:17: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [428]:
############ configuration ################
############################################

CODE, BEHAVIOUR, THREAD, VER = get_config('lora_ducy_config')   ### config stored in libv3/exp_config.txt
TRIAL = 3

base_dir = '../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
log_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR}/trial{TRIAL}'

print(log_path)

#### file to display
trace_file = 0

print('file number:', trace_file)

CODE: lora_ducy
BEHAVIOUR: faulty_data
THREAD: single
VER: 3
../trace_data/lora_ducy/single_thread/version_3/faulty_data/trial3
file number: 0


In [429]:
######### get paths #######################
paths_log, paths_traces, varlist_path, paths_label = get_paths(log_path)

### remove.Ds_store from all lists
paths_log = [x for x in paths_log if '.DS_Store' not in x]
paths_traces = [x for x in paths_traces if '.DS_Store' not in x]
varlist_path = [x for x in varlist_path if '.DS_Store' not in x]
paths_label = [x for x in paths_label if '.DS_Store' not in x]

paths_log.sort()
paths_traces.sort()
varlist_path.sort()

print(paths_log)
print(paths_traces)
print(varlist_path)
print(paths_label)

[]
['../trace_data/lora_ducy/single_thread/version_3/faulty_data/trial3/trace0', '../trace_data/lora_ducy/single_thread/version_3/faulty_data/trial3/trace1', '../trace_data/lora_ducy/single_thread/version_3/faulty_data/trial3/trace2', '../trace_data/lora_ducy/single_thread/version_3/faulty_data/trial3/trace3', '../trace_data/lora_ducy/single_thread/version_3/faulty_data/trial3/trace4', '../trace_data/lora_ducy/single_thread/version_3/faulty_data/trial3/trace5', '../trace_data/lora_ducy/single_thread/version_3/faulty_data/trial3/trace6']
['../trace_data/lora_ducy/single_thread/version_3/faulty_data/trial3/varlist0', '../trace_data/lora_ducy/single_thread/version_3/faulty_data/trial3/varlist1', '../trace_data/lora_ducy/single_thread/version_3/faulty_data/trial3/varlist2', '../trace_data/lora_ducy/single_thread/version_3/faulty_data/trial3/varlist3', '../trace_data/lora_ducy/single_thread/version_3/faulty_data/trial3/varlist4', '../trace_data/lora_ducy/single_thread/version_3/faulty_data/

### Load the existing files

In [430]:
### load combined trace file and varlist
all_content = read_traces(f'/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR}/trace_trial{TRIAL}')
varlist_content = read_json(f'/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR}/varlist_trial{TRIAL}.json')

In [431]:
len(all_content)

1745

In [432]:
len(varlist_content)

86

In [433]:
path_to_content = os.path.dirname(log_path)+f'/trace_trial{TRIAL}'
content_towrite = read_traces(path_to_content)
# print(content_towrite)
# Create an SQLite database (or connect to it if it already exists)
database_url = 'sqlite:///events.db'
engine = create_engine(database_url, echo=True)

# Create all tables in the database
Base.metadata.create_all(engine)

# # Create a configured "Session" class
Session = sessionmaker(bind=engine)

config_settings, config_exists = get_or_create(code_base=CODE, version=VER, behaviour=BEHAVIOUR, trial_num=TRIAL)
print(config_settings, config_exists)

if not config_exists:
    with Session.begin() as session:
        for event_name, event_timestamp in all_content:
            print(event_name, event_timestamp)

            # event_exists = check_event_exists(event_name, event_timestamp, f'{CODE}_{VER}_{BEHAVIOUR}_{TRIAL}')
            # if event_exists:
            #     print(f"Event {event_name} at timestamp {event_timestamp} already exists in the database")
            #     break

            event = Event(name=event_name, timestamp=event_timestamp, config=config_settings)
            session.add(event)
else:
    print("Config already exists in the database")


2024-08-29 11:44:52,478 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-29 11:44:52,482 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("events")
2024-08-29 11:44:52,484 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-29 11:44:52,487 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("file_config")
2024-08-29 11:44:52,488 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-29 11:44:52,489 INFO sqlalchemy.engine.Engine COMMIT
2024-08-29 11:44:52,493 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-29 11:44:52,497 INFO sqlalchemy.engine.Engine SELECT file_config.id AS file_config_id, file_config.unique_name AS file_config_unique_name, file_config.code_base AS file_config_code_base, file_config.version AS file_config_version, file_config.behaviour AS file_config_behaviour, file_config.trial_num AS file_config_trial_num 
FROM file_config 
WHERE file_config.unique_name = ?
 LIMIT ? OFFSET ?
2024-08-29 11:44:52,497 INFO sqlalchemy.engine.Engine [generated in 0.00051s] (

<__main__.File_config object at 0x166e93be0> False
0 44
1 47
2 147
3 430
4 641
5 644
6 649
7 652
8 655
9 657
10 660
11 663
12 665
13 668
14 671
15 673
16 676
17 679
18 681
19 684
20 687
21 689
22 692
23 695
24 697
25 700
26 703
27 706
28 708
29 711
30 714
31 717
32 719
33 722
34 725
35 727
36 730
37 733
38 736
39 739
40 741
41 744
42 747
43 750
44 753
45 755
46 758
47 761
48 764
49 767
50 770
51 773
52 776
53 779
54 785
50 788
51 790
52 793
53 796
54 801
51 804
52 807
53 810
51 812
52 815
53 818
54 823
51 826
52 829
53 832
54 837
51 840
52 843
53 846
54 851
51 853
52 857
53 859
54 864
51 867
52 870
53 873
54 878
51 880
52 884
53 886
54 891
50 894
55 897
54 900
56 968
57 974
58 1277
57 2137
59 2435
60 2438
56 2441
0 2448
61 2458
62 2462
63 2465
38 2468
64 2470
65 2473
66 2476
67 2479
34 2482
68 2484
69 2487
70 2497
71 2500
72 2505
73 2508
22 2522
74 3426
75 3429
61 3432
70 3441
71 3443
72 3448
73 3450
22 3465
74 4368
75 4371
61 4373
70 4382
71 4385
72 4389
73 4391
22 4406
74 5310
75 531

In [434]:
#### query the database


with Session.begin() as session:
    results = session.query(File_config).all()
    for i in results:
        print(i.code_base, i.version, i.behaviour, i.trial_num, i.id)

2024-08-29 11:48:11,145 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-29 11:48:11,149 INFO sqlalchemy.engine.Engine SELECT file_config.id AS file_config_id, file_config.unique_name AS file_config_unique_name, file_config.code_base AS file_config_code_base, file_config.version AS file_config_version, file_config.behaviour AS file_config_behaviour, file_config.trial_num AS file_config_trial_num 
FROM file_config
2024-08-29 11:48:11,150 INFO sqlalchemy.engine.Engine [generated in 0.00079s] ()
theft_protection 3 normal 1 3
theft_protection 3 faulty_data 1 4
theft_protection 3 faulty_data 2 5
theft_protection 3 faulty_data 3 6
mamba2 3 normal 1 14
mamba2 3 normal 2 15
mamba2 3 faulty_data 1 16
mamba2 3 faulty_data 2 17
mamba2 3 faulty_data 4 19
mamba2 3 faulty_data 3 20
lora_ducy 3 normal 1 21
lora_ducy 3 normal 2 22
lora_ducy 3 normal 3 23
lora_ducy 3 normal 4 24
lora_ducy 3 normal 5 25
lora_ducy 3 normal 6 26
lora_ducy 3 normal 7 27
lora_ducy 3 normal 8 28
lora_ducy 3 normal 9 29

In [393]:
# #### delete entries

# to_delete = 29

# with Session.begin() as session:
#     results = session.query(Event).all()
#     file_deleted = False
#     for i in results:
#         if i.file_number == to_delete:
#             print(i.name, i.timestamp, i.file_number, 'deleted')
#             session.delete(i)
#             file_deleted = True

#     if not file_deleted:
#         print('File config does not exist')

# with Session.begin() as session:
#     results = session.query(File_config).all()
#     event_deleted = False
#     for i in results:
#         if i.id == to_delete:
#             print(i.code_base, i.version, i.behaviour, i.trial_num, i.id, 'deleted')
#             session.delete(i)
#             event_deleted = True
            
#     if not event_deleted:
#         print('Event does not exist')

2024-08-29 11:40:51,712 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-29 11:40:51,714 INFO sqlalchemy.engine.Engine SELECT events.id AS events_id, events.name AS events_name, events.timestamp AS events_timestamp, events.file_number AS events_file_number 
FROM events
2024-08-29 11:40:51,715 INFO sqlalchemy.engine.Engine [cached since 5388s ago] ()
0 44 29 deleted
1 47 29 deleted
2 147 29 deleted
3 430 29 deleted
4 641 29 deleted
5 644 29 deleted
6 649 29 deleted
7 652 29 deleted
8 655 29 deleted
9 657 29 deleted
10 660 29 deleted
11 663 29 deleted
12 665 29 deleted
13 668 29 deleted
14 671 29 deleted
15 673 29 deleted
16 676 29 deleted
17 679 29 deleted
18 681 29 deleted
19 684 29 deleted
20 687 29 deleted
21 689 29 deleted
22 692 29 deleted
23 695 29 deleted
24 697 29 deleted
25 700 29 deleted
26 703 29 deleted
27 706 29 deleted
28 708 29 deleted
29 711 29 deleted
30 714 29 deleted
31 717 29 deleted
32 719 29 deleted
33 722 29 deleted
34 725 29 deleted
35 727 29 deleted
36 730